Fuentes: https://medium.com/nlplanet/fine-tuning-distilbert-on-senator-tweets-a6f2425ca50e

#### **Instalar Modulos**

conda install datasets=="2.20.0"

conda install transformers=="4.40.1"

conda install numpy=="1.26.4" # La última versión no funciona bien


In [ ]:
!pip install datasets=="2.20.0"

!pip install transformers=="4.40.1"

!pip install numpy=="1.26.4" # La última versión no funciona bien

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 22.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.5.0 which is incompatible.
torch 2.6.0+cu124

In [ ]:
!pip install transformers

In [ ]:
# Data processing
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from datasets import Dataset,  DatasetDict
#from UA_MDM_LDI_II.tutoriales.utils import plot_confusion_matrix

# Modeling
import torch
from torch.utils.data import DataLoader
from transformers import DistilBertTokenizerFast, DataCollatorWithPadding, AutoModelForSequenceClassification, AdamW, get_scheduler

# Progress bar
from tqdm.auto import tqdm

# Verificamos que CUDA está funcional
torch.cuda.is_available()

False

**Bajamos el modelo**

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

**Armado de los Datasets**

In [ ]:
# Paths
BASE_DIR = '../'
PATH_TO_TRAIN = os.path.join(BASE_DIR, "input/petfinder-adoption-prediction/train/train.csv")
PATH_TO_TEMP_FILES = os.path.join(BASE_DIR, "work/optuna_temp_artifacts")
PATH_TO_OPTUNA_ARTIFACTS = os.path.join(BASE_DIR, "work/optuna_artifacts")

# Parametros y variables
SEED = 42
TEST_SIZE = 0.2
BATCH_SIZE = 64

In [ ]:
# Cargar los datos
train_df = pd.read_csv(PATH_TO_TRAIN)
train_df = train_df[train_df['Description'].notnull()]
train_df['labels'] = train_df["AdoptionSpeed"]

# Dividir los datos usando sklearn
train_df, test_df = train_test_split(train_df, test_size=TEST_SIZE, random_state=SEED, stratify=train_df.AdoptionSpeed)

# Convertir a Dataset
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Combinar en un DatasetDict
dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

# Codificar la columna de etiquetas como clases
dataset = dataset.class_encode_column('labels')

# Hacer una lista de columnas para remover antes de la tokenización
cols_to_remove = [col for col in dataset["train"].column_names if col != 'labels']
print(cols_to_remove)

FileNotFoundError: [Errno 2] No such file or directory: '../input/petfinder-adoption-prediction/train/train.csv'

In [ ]:
# Obtener el objeto ClassLabel del conjunto de datos de entrenamiento
class_label = dataset["train"].features["labels"]

# Obtener las clases originales a partir del objeto ClassLabel
classes = class_label.names
classes

['0', '1', '2', '3', '4']

In [ ]:
# Tokenize and encode the dataset
def tokenize(batch):
    from transformers import DistilBertTokenizerFast
    tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
    tokenized_batch = tokenizer(batch["Description"], padding=True, truncation=True, max_length=512)
    return tokenized_batch

dataset_enc = dataset.map(tokenize, batched=True, remove_columns=cols_to_remove, num_proc=4)

# Set dataset format for PyTorch
dataset_enc.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Check the output
print(dataset_enc["train"].column_names)



Map (num_proc=4):   0%|          | 0/11984 [00:00<?, ? examples/s]/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/

['labels', 'input_ids', 'attention_mask']


In [ ]:
# Instantiate a data collator with dynamic padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create data loaders for to reshape data for PyTorch model
train_dataloader = DataLoader(
    dataset_enc["train"], shuffle=True, batch_size=BATCH_SIZE, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    dataset_enc["test"], batch_size=BATCH_SIZE, collate_fn=data_collator
)

In [ ]:
# Dynamically set number of class labels based on dataset
num_labels = dataset["train"].features['labels'].num_classes
print(f"Number of labels: {num_labels}")

# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
                                                           num_labels=num_labels)

/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Number of labels: 5


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Model parameters
learning_rate = 5e-5
num_epochs = 15

# Create the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Further define learning rate scheduler
num_training_batches = len(train_dataloader)
num_training_steps = num_epochs * num_training_batches
lr_scheduler = get_scheduler(
    "linear",                   # linear decay
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)



/home/aa/miniconda3/envs/ldi2_cuda_trf/lib/python3.9/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


**Miramos el Modelo**

In [ ]:

# Set the device automatically (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Move model to device
model.to(device)


cuda


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

**Entrenamos**

In [ ]:
progress_bar = tqdm(range(num_training_steps))

# Train the model with PyTorch training loop
model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

100%|██████████| 2820/2820 [30:58<00:00,  1.71it/s]

**Obtenemos la kappa base**

In [ ]:
from sklearn.metrics import cohen_kappa_score

# Inicializa listas para almacenar todas las predicciones y etiquetas
all_predictions = []
all_labels = []

# Iteratively evaluate the model and collect predictions and labels
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)

    # Mover predicciones y etiquetas a CPU y convertir a numpy
    all_predictions.extend(predictions.cpu().numpy())
    all_labels.extend(batch["labels"].cpu().numpy())

# Convertir listas a arrays de numpy
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# Calcular Quadratic Weighted Kappa
qwk = cohen_kappa_score(all_labels, all_predictions, weights='quadratic')

print(f"Quadratic Weighted Kappa: {qwk}")


Quadratic Weighted Kappa: 0.22693722211977785


**Predecimos un ejemplo de descripción**

In [ ]:
# Un ejemplo
desc = test_df.iloc[4]['Description']
print(desc)

# Tokenize inputs
inputs = tokenizer(desc, padding=True, truncation=True, return_tensors="pt").to(device) # Move the tensor to the GPU

# Inference model and get logits
outputs = model(**inputs)

# Convert logits to class probabilities
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
probabilities = predictions.detach().cpu().numpy()
predicted_class = np.argmax(probabilities, axis=1)

# Establecer opciones de impresión para evitar la notación científica
np.set_printoptions(suppress=True, formatter={'float_kind': '{:.8f}'.format})
print(probabilities[0])
print(predicted_class)

Cute active little puppy found abondoned and looking for a loving home to live. Please adopt me.....
[0.00010945 0.99926788 0.00034900 0.00022914 0.00004453]
[1]
